In [ ]:
# STEP 0: Check mandatory libraries to run the notebooks
try:
    import nbformat
except ImportError:
    raise ImportError("Mandatory libraries not founds: nbformat -> Run the following command to install it: pip install nbformat")


# STEP 1: Check the current working directory and validate it with an assertion
import os
assert os.path.basename(os.getcwd()) == "GTZAN_Genre_Recognizer", "The script should be starded from root-project folder with GTZAN_Genre_Recognizer name"


# STEP 2: Run all the notebooks in sequence
%run notebooks/01_data_setup.ipynb ##TODO: Test download_data.py and invoke it when dataset is not valid (priority very-low)
%run notebooks/02_features.ipynb
#%run 03_training.ipynb
#%run 04_evaluation.ipynb

c:\Users\rperuffo\OneDrive - Deloitte (O365D)\Desktop\Cr\Machine Learning\GTZAN_Genre_Recognizer\GTZAN_Genre_Recognizer
System detect: Windows
Actual directory:  c:\Users\rperuffo\OneDrive - Deloitte (O365D)\Desktop\Cr\Machine Learning\GTZAN_Genre_Recognizer\GTZAN_Genre_Recognizer
Check if data exists at the following path:  data\genres
All subfolders found
Number of tracks founds:
blues     : 100
classical : 100
country   : 100
disco     : 100
hiphop    : 100
jazz      : 100
metal     : 100
pop       : 100
reggae    : 100
rock      : 100
Total tracks: 1000
